In [2]:
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sqlite3 import Error
from sklearn.ensemble import RandomForestClassifier
import sqlite3
import pickle

#connecting tot he database to load the data. the below database contains the commit messages and their corresponding labels
try:
    conn = sqlite3.connect("training_V2.db")
except Error as e:
    print(e)

#reading the data from the table that contains the labels	
df = pd.read_sql_query('SELECT * FROM filtered', conn)
df.drop(['id'], 1, inplace=True)

# print(df.head())
# this block preprocess the text, removes the stop words and transform the text into vector space using tfidf vectorizer
stemmer = PorterStemmer()
words = stopwords.words("english")
vectorizer = TfidfVectorizer(min_df= 3, stop_words="english", sublinear_tf=True)

df['cleaned'] = df['text'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

# this block is to split the dataset into training and testing set 
X = df['cleaned']
Y = df['class']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20)


pipeline = Pipeline([('vect', vectorizer),
                     ('chi',  SelectKBest(chi2, k=1000)),
                     ('clf', RandomForestClassifier())])

# fitting our model and save it in a pickle for later use
model = pipeline.fit(X_train, y_train)
with open('RandomForest.pickle', 'wb') as f:
    pickle.dump(model, f)

ytest = np.array(y_test)

# confusion matrix and classification report(precision, recall, F1-score)
print(confusion_matrix(ytest, model.predict(X_test)))
print(classification_report(ytest, model.predict(X_test)))
vectorizer = model.named_steps['vect']
chi = model.named_steps['chi']
clf = model.named_steps['clf']

feature_names = vectorizer.get_feature_names()
feature_names = [feature_names[i] for i in chi.get_support(indices=True)]
feature_names = np.asarray(feature_names)

# in this case, I have 5 different classes:

target_names = ['1', '2', '3', '4', '5']
print("top 10 keywords per class:")
for i, label in enumerate(target_names):
    top10 = np.argsort(clf.feature_importances_)[-10:]
    print("%s: %s" % (label, " ".join(feature_names[top10])))


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


[[219   1   0   0   2]
 [  3  53   0   0   1]
 [  0   0  90   0   1]
 [  0   0   0 110   0]
 [  0   0   0   0 196]]
              precision    recall  f1-score   support

           1       0.99      0.99      0.99       222
           2       0.98      0.93      0.95        57
           3       1.00      0.99      0.99        91
           4       1.00      1.00      1.00       110
           5       0.98      1.00      0.99       196

    accuracy                           0.99       676
   macro avg       0.99      0.98      0.99       676
weighted avg       0.99      0.99      0.99       676

top 10 keywords per class:
1: cloudstack design refactor bug duplic structur perform improv fix featur
2: cloudstack design refactor bug duplic structur perform improv fix featur
3: cloudstack design refactor bug duplic structur perform improv fix featur
4: cloudstack design refactor bug duplic structur perform improv fix featur
5: cloudstack design refactor bug duplic structur perform improv

c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
